## Completely unprocessed data
## Only one hot encoding categorical features

In [2]:
import numpy as np
np.random.seed(123)

import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from scipy.sparse import csr_matrix, hstack
from scipy.stats import skew
from scipy.stats import boxcox
from scipy import stats

In [26]:
print "Data Preprocessing Begins..."
## Read data
train = pd.read_csv('../data/train.csv')

## Response and IDs
y = train['loss'].values
train = train.drop(['id', 'loss'], 1)

f_cat = [f for f in train.columns if 'cat' in f]
for f in f_cat:
    dummy = pd.get_dummies(train[f].astype('category'))
    train[f] = dummy
    print train[f].unique()

print "Data Preprocessing Ends..."

Data Preprocessing Begins...
[1 0]
[0 1]
[1 0]
[0 1]
[1 0]
[1 0]
[1 0]
[1 0]
[0 1]
[1 0]
[0 1]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[0 1]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[1 0]
[1 0]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[1 0]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[0 1]
[1 0]
[0 1]
[0 1]
Data Preprocessing Ends...


In [33]:
print "Data Preprocessing Begins..."
## Read data
train = pd.read_csv('../data/train.csv')

## Response and IDs
y = train['loss'].values
train = train.drop(['id', 'loss'], 1)

f_cat = [f for f in train.columns if 'cat' in f]
for f in f_cat:
    dummy = pd.get_dummies(train[f].astype('category'))
    print type(dummy), dummy.shape
    train = pd.concat([train, dummy], 1)
    train = train.drop(f, 1)

print "Data Preprocessing Ends..."

Data Preprocessing Begins...
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.frame.DataFrame'> (188318, 2)
<class 'pandas.core.f

In [34]:
print train.shape
print y.shape

(188318, 1153)
(188318,)


In [35]:
train.head(20)

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,...,P,Q,R,S,T,U,V,W,X,Y
0,0.726300,0.245921,0.187583,0.789639,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,...,0,0,0,0,0,0,0,0,0,0
1,0.330514,0.737068,0.592681,0.614134,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,...,0,0,0,0,0,0,0,0,0,0
2,0.261841,0.358319,0.484196,0.236924,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,...,0,0,0,0,0,0,0,0,0,0
3,0.321594,0.555782,0.527991,0.373816,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,...,0,0,0,0,0,0,0,0,0,0
4,0.273204,0.159990,0.527991,0.473202,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,...,0,0,0,0,0,0,0,0,0,0
5,0.546670,0.681761,0.634224,0.373816,0.302678,0.364464,0.401162,0.26847,0.46226,0.50556,...,0,0,0,0,0,0,0,0,0,0
6,0.471447,0.737068,0.613660,0.189137,0.295397,0.381515,0.363768,0.24564,0.40455,0.47225,...,0,0,0,0,0,0,0,0,0,0
7,0.826591,0.488789,0.263570,0.623770,0.473767,0.867021,0.583389,0.90267,0.84847,0.80218,...,0,0,0,0,0,0,0,0,0,0
8,0.330514,0.555782,0.440642,0.473202,0.281143,0.628534,0.384099,0.61229,0.38249,0.51111,...,0,0,0,0,0,0,0,0,0,0
9,0.726300,0.358319,0.356819,0.802892,0.310061,0.713343,0.469223,0.30260,0.67135,0.83510,...,0,0,0,0,0,0,0,0,0,0


# Ridge Regression

In [36]:
from sklearn.linear_model import Ridge

In [37]:
%%time

ridge = Ridge()
ridge_params = {'alpha': [1]}
ridge_grid = GridSearchCV(ridge, ridge_params, cv=5, verbose=10, scoring='neg_mean_absolute_error')
ridge_grid.fit(train, y)
ridge_score = ridge_grid.cv_results_
print ridge_score

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] alpha=1 .........................................................
[CV] ...................... alpha=1, score=-1307.440420, total=   7.3s
[CV] alpha=1 .........................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    8.3s remaining:    0.0s


[CV] ...................... alpha=1, score=-1286.651574, total=   7.4s
[CV] alpha=1 .........................................................


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:   16.6s remaining:    0.0s


[CV] ...................... alpha=1, score=-1302.002107, total=   7.4s
[CV] alpha=1 .........................................................


...

[CV] ...................... alpha=1, score=-1308.157692, total=   7.4s
[CV] alpha=1 .........................................................


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   33.3s remaining:    0.0s


[CV] ...................... alpha=1, score=-1296.228012, total=   7.4s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   41.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:   41.7s finished


{'rank_test_score': array([1], dtype=int32), 'split4_test_score': array([-1296.22801165]), 'mean_score_time': array([ 0.23095741]), 'std_test_score': array([ 7.97846863]), 'std_train_score': array([ 2.48296248]), 'split1_train_score': array([-1293.19128078]), 'split0_test_score': array([-1307.44042007]), 'mean_test_score': array([-1300.09593872]), 'std_score_time': array([ 0.00741198]), 'split2_train_score': array([-1286.63343551]), 'param_alpha': masked_array(data = [1],
             mask = [False],
       fill_value = ?)
, 'split0_train_score': array([-1288.66694076]), 'params': ({'alpha': 1},), 'std_fit_time': array([ 0.04424911]), 'split4_train_score': array([-1290.99178817]), 'split2_test_score': array([-1302.00210671]), 'split3_test_score': array([-1308.15769231]), 'mean_train_score': array([-1289.29581781]), 'mean_fit_time': array([ 7.15296397]), 'split3_train_score': array([-1286.99564385]), 'split1_test_score': array([-1286.6515742])}
CPU times: user 1min 38s, sys: 14 s, total

In [38]:
# alpha = 1 is the best

print ridge_grid.best_params_
print ridge_score['mean_test_score']
print ridge_score['params']

{'alpha': 1}
[-1300.09593872]
({'alpha': 1},)
